In [1]:
import torch
import torch.optim as optim
from torch.nn.functional import cross_entropy

from src.RandmanFunctions import read_randman10_dataset
from src.Models import RandmanSNN
from src.EvolutionAlgorithms.EvolutionStrategy import ESModel
from src.Training import train_loop_snn

device = 'cpu'

/home/wyx/miniconda3/envs/snn/lib/python3.12/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


## 2 Training SNN for Randman

In [ ]:
import wandb
device = 'cpu'

def train_snn():   
    run_name = 'sanity_check'
    config = { # Dataset:
              'nb_input' : 100,
              'nb_output' : 10,  
              'nb_steps' : 50,
              'nb_data_samples': 1000,
              # SNN:
              'nb_hidden' : 10,
              'learn_beta' : False,             
              # Evolution Strategy:
              'nb_model_samples' : 20, 
              'mirror' : True,
              # Training: 
              'std' : 0.05,
              'epochs' : 50, 
              'batch_size' : 256,
              # Optimization:
              'loss': 'cross-entropy',
              'optimizer' : 'Adam',
              'lr' : 0.01,
              'regularization':'none'}
    with torch.no_grad(), wandb.init(entity = 'DarwinNeuron', project = 'ES-Randman10', name=run_name, config=config) as run:  
        # initialize Evolution Strategy instance
        es_model = ESModel(RandmanSNN, run.config.nb_input, run.config.nb_hidden, run.config.nb_output, 0.95, sample_size=run.config.nb_model_samples, param_std = run.config.std, Optimizer=optim.Adam, lr=run.config.lr, device=device, mirror=run.config.mirror)
        
        # load dataset
        train_loader, val_loader = read_randman10_dataset('data/randman_10_dataset.pt', batch_size=run.config.batch_size)
        
        # epochs
        for epoch in range(run.config.epochs):
            print(f"Epoch {epoch}\n-------------------------------")
            
            # train the model
            train_loop_snn(es_model, train_loader, val_loader, cross_entropy, device, run)
    
train_snn() 

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yixing to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 0
-------------------------------
batch 0, loss: 2.380389, accuracy: 10.9%
Test Error: 
Accuracy: 9.4%, Avg loss: 2.374867 

batch 1, loss: 2.382493, accuracy: 9.4%
Test Error: 
Accuracy: 9.4%, Avg loss: 2.366478 

batch 2, loss: 2.362677, accuracy: 7.4%
Test Error: 
Accuracy: 8.6%, Avg loss: 2.361515 

batch 3, loss: 2.353785, accuracy: 9.0%
Test Error: 
Accuracy: 9.0%, Avg loss: 2.355470 

batch 4, loss: 2.349546, accuracy: 7.0%
Test Error: 
Accuracy: 8.2%, Avg loss: 2.348285 

batch 5, loss: 2.330541, accuracy: 14.5%
Test Error: 
Accuracy: 8.9%, Avg loss: 2.343625 

batch 6, loss: 2.336818, accuracy: 8.6%
Test Error: 
Accuracy: 9.2%, Avg loss: 2.339010 

batch 7, loss: 2.333914, accuracy: 9.0%
Test Error: 
Accuracy: 9.1%, Avg loss: 2.332818 

batch 8, loss: 2.336750, accuracy: 9.8%
Test Error: 
Accuracy: 9.0%, Avg loss: 2.330187 

batch 9, loss: 2.331414, accuracy: 8.2%
Test Error: 
Accuracy: 9.2%, Avg loss: 2.327248 

batch 10, loss: 2.313446, accuracy: 10.5%
Test Error: 
Acc